# Agents to Research and Write an Article

In [6]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq==0.0.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.2 MB/s eta 0:00:00


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

In [5]:
import os

from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [7]:
from langchain_groq import ChatGroq

llm = llm = ChatGroq(
    model="gemma-7b-it",
    groq_api_key=os.environ["GROQ_API_KEY"],
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
## Content Plan: Artificial Intelligence

**Final Answer:**

**I. Introduction (200 words)**

* Definition of Artificial Intelligence (AI)
* Brief history of AI development
* Importance of AI in the modern world
* Overview of current AI trends and applications

**II. Key Trends in AI (600 words)**

* **Generative AI:** Text and code generation, image and video creation
* **Reinforcement Learning:** Machine learning algorithms learn from experience and improve outcomes
* **Natural Language Processing:** Advancements in understanding and generating human l

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

**Artificial Intelligence: Shaping the Future**

Artificial Intelligence (AI) has emerged as a transformative technology, revolutionizing industries and impacting lives across the globe. From language translation to healthcare diagnostics, AI algorithms have demonstrated remarkable capabilities in understanding and mimicking human intelligence. This remarkable progress has led to diverse applications across sectors, shaping the future of numerous industries.

**Key Trends Shaping AI's Trajectory**

AI advancements are driven by innovative trends that enable remarkable capabilities. Generative AI empowers the creation of realistic text and images, while Reinforcement Learning algorithms learn from data and make optimal decisions over time. Natural Language Processing has advanced significantly, allowing for deeper understanding and generation of human language. Additionally, Edge AI brings AI processing closer to data sources, improving efficiency and accessibility. Finally, Explainable AI emphasizes the need for transparency and accountability in AI models.

**Leading Players and Promising Innovations**

Leading technology giants like Google, Amazon, and Microsoft are driving AI innovation. Their platforms and tools, such as TensorFlow and SageMaker, are widely used for machine learning model development and deployment. Additionally, numerous startups like OpenAI and Anthropic are shaping the future of AI with their groundbreaking technologies.

**AI Applications Across Industries**

AI applications are expanding rapidly across industries. Recent breakthroughs include AI-powered drug discovery, robots aiding disaster relief efforts, and AI-driven chatbots mimicking human interaction. Governments are utilizing AI for surveillance, predictive analytics, and citizen services.

**Ethical Considerations and the Future**

AI offers transformative potential across industries, but ethical considerations must be addressed. As AI technology evolves, ensuring fairness, accountability, and transparency will be crucial. Responsible implementation of AI will be essential for maximizing its benefits and mitigating potential risks.

**Conclusion**

AI is poised to further revolutionize the world, empowering industries and shaping the future. By embracing its potential while addressing ethical considerations, we can unlock the transformative power of AI to address complex problems and create a better future.

**Call to Action**

To explore the potential of AI:

* Stay updated with AI news and publications
* Visit the websites of leading AI companies
* Attend AI conferences and workshops
* Invest in AI-powered products and services
* Join online communities and forums to connect with AI experts and enthusiasts